In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

In [2]:
# Load the dataset
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
 # Preprocess the data - drop irrelevant data
data = df.drop(['RowNumber','CustomerId','Surname' ],axis=1)

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Encoding for Categorical column - gender, country

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender .fit_transform(data['Gender'])

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# one hot encoding on geography

from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])

In [9]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray() , columns= one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [12]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [13]:
# Combine One hot encoded columns with the original data

data = pd.concat([data.drop(['Geography'],axis=1), geo_encoded_df],axis=1)

In [14]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
# Now I have to save these files as pickle file(the encoded one)  
# Save the encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl','wb')as file:
    pickle.dump(one_hot_encoder_geo, file)

 

In [16]:
# Divide the dataset into independent and dependent dataset

X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data

X_train, X_test , y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Scale down the feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [17]:
# Now I will also save my scaler file as pickle file

with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

# ANN Implementation

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  # required for Neural Network
from tensorflow.keras.layers import Dense   # to define nodes in a layer
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 


In [19]:
# Build ANN model

# if I do X_train.shape that many will be my inputs (input nodes) and create it using Dense

model = Sequential([
    # 64 = total neurons, 
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),    # HL1 connected with input layer
    Dense(32, activation='relu'),   # second layer now it already knows it will be connected to input so need to mention it
    Dense(1,activation='sigmoid')    # output layer
])

/Users/rishabh/ML/Projects/Churn Prediction Using ANN/avenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-02-26 11:44:33.846932: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2026-02-26 11:44:33.846968: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-02-26 11:44:33.846973: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2026-02-26 11:44:33.847005: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-26 11:44:33.847015: I tensorflow/core/common_runtime/pluggable_device/plugga

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# To do the forward and backward prop. we need to compile model
# so we will compile model


opt = tf.keras.optimizers.Adam(learning_rate = 0.01)
# apart from Adam other optimizers can be used

loss = tf.keras.losses.BinaryCrossentropy()
 

In [22]:
# There are other ways to use optimizers

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'] )
# adam has a fixed learning rate you can use keras if you want your own learning rate

In [23]:
# Setup the Tensorboard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [24]:
# Setup early stopping
# I can train my model for many epochs but lets say my model is at best after 10 epochs and then it is varying by just
# 1-2 % then its better to stop and not run 100 or more epochs
# We monitor and if loss is not decreasing after some time so apply early stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# patience means wait for that many epochs to make a decision


Training the Model

In [25]:
history = model.fit(
    X_train,y_train,
    validation_data =(X_test, y_test), 
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


2026-02-26 11:44:34.772936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.7815 - loss: 0.5353 - val_accuracy: 0.7965 - val_loss: 0.5448
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7291 - loss: 1.6827 - val_accuracy: 0.6435 - val_loss: 3.7293
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7050 - loss: 5.2389 - val_accuracy: 0.7930 - val_loss: 5.6226
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7164 - loss: 10.3592 - val_accuracy: 0.7975 - val_loss: 4.9509
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7355 - loss: 9.5189 - val_accuracy: 0.6890 - val_loss: 32.1397
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7222 - loss: 19.9359 - val_accuracy: 0.7615 - val_loss: 18.4083


In [27]:
# save the model, .h5 model is compatible with keras
model.save('model.h5')

Load Tensorboard Extension

In [28]:
%load_ext tensorboard

In [30]:
%tensorboard  --logdir logs/fit20260226-114434/

# Doing the Prediction
Load the pkl file

In [31]:
# in the next file prediction.ipynb

